In [17]:
import boto3
import json
from datetime import datetime
import os


AWS_REGION = "us-east-2"
BUCKET_NAME = f'employee-photo-bucket-{datetime.now().strftime("%d-%m-%Y-%H-%M-%S")}'
S3_CLIENT = boto3.client("s3", region_name=AWS_REGION)
S3_RESOURSE = boto3.resource("s3", region_name=AWS_REGION)
IAM_CLIENT = boto3.client('iam', region_name=AWS_REGION)

## Create Role

In [ ]:
AssumeRolePolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "ec2.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

IAM_CLIENT.create_role(
                        RoleName='EmployeeDirectoryAppRole',
                        AssumeRolePolicyDocument=json.dumps(AssumeRolePolicy)
                    )

## Embed an inline policy in the role

In [ ]:
InlinePolicy = {
    "Statement": [
        {
            "Condition": {
                "StringEquals": {
                    "ec2:osuser": "ec2-user"
                }
            },
            "Action": [
                "ec2-instance-connect:SendSSHPublicKey"
            ],
            "Resource": [
                "arn:aws:ec2:*:*:instance/*"
            ],
            "Effect": "Allow"
        }
    ]
}

IAM_CLIENT.put_role_policy(
                            RoleName='EmployeeDirectoryAppRole',
                            PolicyName='DirectoryWebApplicationSSHPolicy',
                            PolicyDocument=json.dumps(InlinePolicy)
                            )

## Creating S3 Bucket

In [18]:
bucket = S3_CLIENT.create_bucket(Bucket=BUCKET_NAME, 
                                CreateBucketConfiguration={'LocationConstraint':AWS_REGION}
                                )
bucket

{'ResponseMetadata': {'RequestId': '5KY2YVDEH1GVQ4D1',
  'HostId': 'SLfl902Q7qcGgE3wJuHcrIQGlmEBKWVCfUS4hf8M+Ig0W+j7ObMeNrpIo99VFWPTNi6y3W0g25Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SLfl902Q7qcGgE3wJuHcrIQGlmEBKWVCfUS4hf8M+Ig0W+j7ObMeNrpIo99VFWPTNi6y3W0g25Y=',
   'x-amz-request-id': '5KY2YVDEH1GVQ4D1',
   'date': 'Sat, 05 Feb 2022 17:05:46 GMT',
   'location': 'http://employee-photo-bucket-05-02-2022-19-06-12.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://employee-photo-bucket-05-02-2022-19-06-12.s3.amazonaws.com/'}

In [29]:
ACCOUNT_NUMBER = "593222115585"
Policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
            "Sid": "AllowS3ReadAccess",
            "Effect": "Allow",
            "Principal": {
                "AWS": f"arn:aws:iam::{ACCOUNT_NUMBER}:role/EmployeeDirectoryAppRole"
            },
            "Action": "s3:*",
            "Resource": [
                f"arn:aws:s3:::{BUCKET_NAME}",
                f"arn:aws:s3:::{BUCKET_NAME}/*"
            ]
            }
        ]
        }

In [30]:
response = S3_CLIENT.put_bucket_policy(
    Bucket=BUCKET_NAME,
    Policy=json.dumps(Policy),
    ExpectedBucketOwner=ACCOUNT_NUMBER
)

## Uploading a file to S3 Bucket

In [37]:
keys = [file for file in os.listdir("img/") if file.endswith(".png")]
keys

['employee-1.png',
 'employee-10.png',
 'employee-2.png',
 'employee-3.png',
 'employee-4.png',
 'employee-5.png',
 'employee-6.png',
 'employee-7.png',
 'employee-8.png',
 'employee-9.png']

In [38]:
for Key in keys:
    S3_CLIENT.upload_file(f'img/{Key}', BUCKET_NAME, Key)

## Deleting non-empty S3 Bucket

In [53]:
for bucket in S3_RESOURSE.buckets.all():
    print(bucket.name)

c20300a265023l850106t1w593222115585-labbucket-13tw1hdre6l1j
employee-photo-bucket-05-02-2022-19-06-12


In [54]:
for object in S3_RESOURSE.Bucket(BUCKET_NAME).objects.all():
    print('Deleting.. ' + object.key)
    object.delete()
S3_RESOURSE.Bucket(BUCKET_NAME).delete()

Deleting.. employee-1.png
Deleting.. employee-10.png
Deleting.. employee-2.png
Deleting.. employee-3.png
Deleting.. employee-4.png
Deleting.. employee-5.png
Deleting.. employee-6.png
Deleting.. employee-7.png
Deleting.. employee-8.png
Deleting.. employee-9.png


{'ResponseMetadata': {'RequestId': 'AFYJKNBGDC1KVYED',
  'HostId': 'UX8GT7608agdXF/c1hAGnHUVms2EptJFnSBT7xehU5e335aG6fbu+Cc/LqXSKJ1BnzjmQXOk53g=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'UX8GT7608agdXF/c1hAGnHUVms2EptJFnSBT7xehU5e335aG6fbu+Cc/LqXSKJ1BnzjmQXOk53g=',
   'x-amz-request-id': 'AFYJKNBGDC1KVYED',
   'date': 'Sat, 05 Feb 2022 18:43:42 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [ ]:
IAM_CLIENT.delete_role(RoleName='EmployeeDirectoryAppRole')